In [268]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.options.display.max_rows = 1000 
pd.options.display.max_columns = 50

In [269]:
# url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.csv"
# disasterDataDf = pd.read_csv(url, index_col='state')

data = pd.read_csv('data.csv')

In [270]:
data = data.drop(['id','lastRefresh','hash','declarationRequestNumber','placeCode','fipsCountyCode','fipsStateCode','hmProgramDeclared','paProgramDeclared','iaProgramDeclared','ihProgramDeclared','fyDeclared','disasterNumber','disasterNumber','declarationType','incidentBeginDate','incidentEndDate','designatedArea'],axis = 1).fillna('')
data.head(20)

,femaDeclarationString,state,declarationDate,incidentType,declarationTitle,disasterCloseoutDate
0,DR-7-MA,MA,1953-06-11T00:00:00.000Z,Tornado,TORNADO,1956-06-01T00:00:00.000Z
1,DR-9-TX,TX,1953-06-19T00:00:00.000Z,Flood,FLOOD,1958-01-01T00:00:00.000Z
2,DR-8-IA,IA,1953-06-11T00:00:00.000Z,Flood,FLOOD,1955-11-01T00:00:00.000Z
3,DR-11-NH,NH,1953-07-02T00:00:00.000Z,Fire,FOREST FIRE,1956-02-01T00:00:00.000Z
4,DR-6-MI,MI,1953-06-09T00:00:00.000Z,Tornado,TORNADO,1956-03-30T00:00:00.000Z
5,DR-5-MT,MT,1953-06-06T00:00:00.000Z,Flood,FLOODS,1955-12-01T00:00:00.000Z
6,DR-3-LA,LA,1953-05-29T00:00:00.000Z,Flood,FLOOD,1960-02-01T00:00:00.000Z
7,DR-4-MI,MI,1953-06-02T00:00:00.000Z,Tornado,TORNADO,1956-02-01T00:00:00.000Z
8,DR-1-GA,GA,1953-05-02T00:00:00.000Z,Tornado,TORNADO,1954-06-01T00:00:00.000Z
9,DR-2-TX,TX,1953-05-15T00:00:00.000Z,Tornado,TORNADO & HEAVY RAINFALL,1958-01-01T00:00:00.000Z


In [271]:
def get_closeout_duration(df):
    df = df.applymap(lambda x: datetime.strptime(x[:x.index('T')], '%Y-%m-%d') if x != '' else datetime.today().date())
    return df['disasterCloseoutDate'] - df['declarationDate']

In [272]:
data[['declarationDate', 'disasterCloseoutDate']] = data[['declarationDate', 'disasterCloseoutDate']].applymap(lambda date: datetime.strptime(date[:date.index('T')], '%Y-%m-%d') if date != '' else datetime.today().date())
data['closeoutDuration'] = data['disasterCloseoutDate'] - data['declarationDate']

In [273]:
data_cleaned = data.groupby(['femaDeclarationString']).head(1).set_index(['incidentType','state'])

In [274]:
data_cleaned.groupby(['incidentType','state']).head(1)\
    .sort_values(['incidentType','closeoutDuration'],ascending=[True,False])\
    .groupby(['incidentType']).head(5)\
    .rename(columns={'closeoutDuration': 'Max Closeout Time'})

femaDeclarationString declarationDate  \
incidentType     state                                         
Biological       WA               EM-3427-WA      2020-03-13   
                 CA               EM-3428-CA      2020-03-13   
                 FL               EM-3432-FL      2020-03-13   
                 IL               EM-3435-IL      2020-03-13   
                 VT               EM-3437-VT      2020-03-13   
Chemical         WV               EM-3366-WV      2014-01-10   
Coastal Storm    TX               DR-1379-TX      2001-06-09   
                 CA                DR-677-CA      1983-02-09   
                 LA               DR-1380-LA      2001-06-11   
                 MH               DR-1040-MH      1994-10-06   
                 MA                DR-920-MA      1991-11-04   
Dam/Levee Break  CO                DR-379-CO      1973-05-08   
                 CA                DR-161-CA      1963-12-21   
                 MI               EM-3525-MI      2020-05-21   
Drought          MH               DR-1210-MH      1998-03-20   
                 FM               DR-1213-FM      1998-04-03   
                 MN               EM-3013-MN      1976-06-17   
                 WI               EM-3014-WI      1976-06-17   
                 SD               EM-3015-SD      1976-06-17   
Earthquake       CA                DR-299-CA      1971-02-09   
                 AS               DR-1859-AS      2009-09-29   
                 VA               DR-4042-VA      2011-11-04   
                 NY               DR-1415-NY      2002-05-16   
                 GU               DR-1397-GU      2001-12-05   
Fire             NY               DR-1391-NY      2001-09-11   
                 FL               DR-1223-FL      1998-06-18   
                 NM               DR-1329-NM      2000-05-13   
                 WY               FM-2315-WY      2000-07-31   
                 TX               DR-1624-TX      2006-01-11   
Fishing Losses   CA               DR-1038-CA      1994-09-13   
                 WA               DR-1037-WA      1994-08-02   
                 OR               DR-1036-OR      1994-08-02   
                 ME                DR-356-ME      1972-09-28   
                 MA                DR-357-MA      1972-09-28   
Flood            SC                DR-881-SC      1990-10-22   
                 UT                DR-680-UT      1983-04-30   
                 CT                DR-661-CT      1982-06-14   
                 MP                DR-562-MP      1978-08-18   
                 MH                DR-932-MH      1992-02-07   
Freezing         AZ               DR-4104-AZ      2013-03-05   
                 ME               DR-1468-ME      2003-05-14   
                 CA                DR-894-CA      1991-02-11   
                 MI               DR-1028-MI      1994-05-10   
                 VA                DR-525-VA      1977-01-26   
Human Cause      FL               EM-3079-FL      1980-05-06   
                 OK               DR-1048-OK      1995-04-26   
                 NY                DR-984-NY      1993-04-02   
Hurricane        AS                DR-785-AS      1987-01-24   
                 KY               EM-3231-KY      2005-09-10   
                 PA                DR-745-PA      1985-10-08   
                 MD               DR-1139-MD      1996-09-17   
                 DE               DR-1297-DE      1999-09-21   
Mud/Landslide    NC               DR-4103-NC      2013-03-01   
                 OH                DR-390-OH      1973-06-04   
                 WA               DR-1255-WA      1998-10-16   
                 HI               DR-4282-HI      2016-10-06   
                 WV               DR-4359-WV      2018-04-17   
Other            TX               EM-3171-TX      2003-02-01   
                 PR               EM-3124-PR      1996-11-21   
                 KS               EM-3126-KS      1998-06-09   
                 RI               EM-3123-RI      1996-

In [275]:
data_cleaned.drop(['femaDeclarationString','disasterCloseoutDate','disasterCloseoutDate','declarationDate'],axis=1)\
    .groupby(['incidentType','state'])\
    .apply(lambda durations: durations.mean())\
    .sort_values(['incidentType','closeoutDuration'],ascending=[True,False])\
    .groupby(['incidentType']).head(10)\
    .rename(columns={'closeoutDuration': 'Average Closeout Time'})\
    .loc[['Tornado']]

Average Closeout Time
incidentType state                      
Tornado      FL       4832 days 00:00:00
             SC       3860 days 00:00:00
             MN       3799 days 00:00:00
             SD       3491 days 00:00:00
             NC       3044 days 12:00:00
             MD       2947 days 00:00:00
             NY       2885 days 00:00:00
             KY       2794 days 04:48:00
             CT       2536 days 16:00:00
             MA       2383 days 00:00:00